In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from WelfareMeasures import *
from functions_njit import tax_rate_fct
from bernoulli_distribution import Bernoulli
from help_functions_non_njit import *
from graph_format import *
# load local model file and initialize model class
from model import ModelClass
from scipy.optimize import minimize

In [ ]:
model = ModelClass() # set a few number of periods.

par = model.par
sol = model.sol
sim = model.sim
par.opt_tol = 1e-6
par.opt_maxiter = 1000

%time model.solve(do_print=False)
%time model.simulate()

In [ ]:
ret_age = 49
increasing_RPA = []

for first_age in range(30, ret_age + 1):
    early_age_lag = ret_age- first_age-4 #skal være 4 år før SPA

    theta = [ret_age, first_age, early_age_lag]
    theta_names = ['retirement_age', 'first_retirement', 'early_benefits_lag', ]

    og_model, new_model = make_new_model(model, theta, theta_names, do_print=False)
    consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= False, the_method = 'brentq')

    pi_cum = np.cumprod(og_model.par.pi)

    total_margin_og = np.nanmean(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement])
    total_margin_new = np.nanmean(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement])
    total_margin = (total_margin_new - total_margin_og)

    increasing_RPA.append([ret_age, first_age, early_age_lag+ first_age, total_margin, consumption_eq])
    print("Done with first_age", first_age)

np.savetxt('increasing_RPA.txt', increasing_RPA)



In [ ]:
first_retirement = 30
early_age_lag = 1
increasing_SPA_fix_RPA_EPA = []

for ret_age in range(first_retirement +early_age_lag, 50):

    theta = [ret_age, first_age, early_age_lag]
    theta_names = ['retirement_age', 'first_retirement', 'early_benefits_lag', ]

    og_model, new_model = make_new_model(model, theta, theta_names, do_print=False)
    consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= False, the_method = 'brentq')

    pi_cum = np.cumprod(og_model.par.pi)

    total_margin_og = np.nanmean(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement])
    total_margin_new = np.nanmean(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement])
    total_margin = (total_margin_new - total_margin_og)

    increasing_SPA_fix_RPA_EPA.append([ret_age, first_age, early_age_lag+ first_age, total_margin, consumption_eq])
    print("Done with first_age", ret_age)

np.savetxt('increasing_SPA_fix_RPA_EPA.txt', increasing_SPA_fix_RPA_EPA)




In [ ]:

first_retirement = 30
early_age_lag = 1
increasing_SPA_fix_RPA_EPA = []

for ret_age in range(first_retirement +early_age_lag, 50):
    print(ret_age, first_retirement, early_age_lag)
    theta = [ret_age, first_retirement, early_age_lag]
    theta_names = ['retirement_age', 'first_retirement', 'early_benefits_lag', ]

    og_model, new_model = make_new_model(model, theta, theta_names, do_print=False)
    consumption_eq = find_consumption_equivalence(og_model, new_model, do_print= False, the_method = 'brentq')

    pi_cum = np.cumprod(og_model.par.pi)

    total_margin_og = np.nanmean(pi_cum[:og_model.par.last_retirement] * og_model.sim.h[:, :og_model.par.last_retirement])
    total_margin_new = np.nanmean(pi_cum[:og_model.par.last_retirement] * new_model.sim.h[:, :og_model.par.last_retirement])
    total_margin = (total_margin_new - total_margin_og)

    increasing_SPA_fix_RPA_EPA.append([ret_age, first_retirement, early_age_lag+ first_retirement, total_margin, consumption_eq])
    print("Done with first_age", ret_age)

np.savetxt('increasing_SPA_fix_RPA_EPA_RPA_at_30.txt', increasing_SPA_fix_RPA_EPA)




In [ ]:
df_SPA = pd.read_csv('increasing_SPA_fix_RPA_EPA_RPA_at_30.txt', sep='\s+', header=None)

# Assign column names (optional)
df_SPA.columns = ['SPA', 'RPA', 'EPA', 'LS', 'CE']
# df['EPA'] = df['EPA']+df['RPA']
type_dict = {'SPA': int, 'RPA':int, 'EPA':int}
df_SPA = df_SPA.astype(type_dict)
df_SPA[['LS', 'CE']] = df_SPA[['LS', 'CE']]*100
df_SPA['LS_relativ'] =df_SPA['LS'] /total_margin_og
df_SPA[['SPA', 'RPA','EPA']] = df_SPA[['SPA', 'RPA', 'EPA']]+30